<a href="https://colab.research.google.com/github/FiddiCoder/-FIA-Project/blob/main/ml_pipeline/Movie_Recommender_DecisionTree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [248]:
#@title Import Librerie
import pandas as pd
import numpy as np
import re
from tqdm import tqdm_notebook as tqdm
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
import os
import tarfile
from six.moves import urllib
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn import datasets, linear_model,tree
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [249]:
#@title Download e formattazione dataset film

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/FiddiCoder/-FIA-Project/main/"
FILE_PATH = os.path.join("dataset")
FILE_URL1 = DOWNLOAD_ROOT + "dataset/movies2"

def fetch_file_data1(file_url1=FILE_URL1, file_path1=FILE_PATH):
    os.makedirs(file_path1, exist_ok=True)
    csv_path1 = os.path.join(file_path1, "movies2")
    urllib.request.urlretrieve(file_url1, csv_path1)

fetch_file_data1()
datapath = os.path.join("dataset","")

with open('./dataset/movies2', encoding = "ISO-8859-1") as content:
    colonne = ['movie_id', 'movie_title', 'release_date', 'video_release_date', 'imdb_url']
    generi = ['unknown', 'action', 'adventure', 'animation', 'children', 'comedy', 'crime', 'documentary', 'drama', 'fantasy',
          'film-noir',  'horror', 'musical', 'mystery', 'romance', 'sci-fi', 'thriller', 'war', 'western']
    colonneG = colonne + generi
    movies = pd.DataFrame(columns=colonneG)
    i = 0
    for x in content:
        x = x.split("|")
        x[-1] = x[-1][:-1]
        if x[1][-1] == ' ':
            x[1] = x[1][:-1]
        movies.loc[i] = [word if word!='' else "empty" for word in x]
        i = i + 1
movies['movie_id'] = movies['movie_id'].astype('int64')
movies[generi] = movies[generi].astype('category')



In [250]:
#@title Data cleaning: Tabelle con feature inutili
movies = movies.drop('video_release_date', axis=1)
movies = movies.drop('release_date', axis=1)
movies = movies.drop('imdb_url', axis=1)


In [ ]:
#@title Inserimento colonna target con relativo riempimento
movies['like'] = 0

for i in range(1600):
  if((i % 3) == 0):
    movies.loc[i, 'like'] = 1
  if((i % 8) == 0):
    movies.loc[i , 'like'] = 2
  if((i % 7) == 0):
     movies.loc[i , 'like'] = 3
  if((i % 5) == 0):
    movies.loc[i , 'like'] = 4
  if((i % 6) == 0):
    movies.loc[i , 'like'] = 5


movies.head(100)

In [ ]:
#@title Primo Test: Decision tree

#@markdown Risultati scarsi.

#@markdown Abbiamo pensato di joinare il dataset dei film con il dataset degli utenti che contiene a sua volta una variabile target (rating) con valori opportunamente riempiti e non inseriti manualmente da noi
X = movies.iloc[:,2:20]
print(X.columns)
y = movies.iloc[:,21:]
print(y.columns)
X_train, X_test, y_train, y_test = train_test_split(X, y,
test_size=0.20, random_state=0)

dtree = tree.DecisionTreeClassifier()
modello = dtree.fit(X_train.values, y_train)

for i in range(1, len(movies)):
    movie = movies.iloc[i, 1]
    if(movie == 'Turbo: A Power Rangers Movie (1997)'):
        movie_1_ind = movies.iloc[i,2:20]
    if(movie == 'Dead Man Walking (1995)'):
        movie_2_ind = movies.iloc[i,2:20]
    if (movie == 'Chasing Amy (1997)'):
        movie_3_ind = movies.iloc[i,2:20]

print("prediction for Turbo: A Power Rangers Movie (1997)")
print(dtree.predict([movie_1_ind]))
print("prediction for Dead Man Walking (1995)")
print(dtree.predict([movie_2_ind]))
print("prediction for Chasing Amy (1997)")
print(dtree.predict([movie_3_ind]))
print("\n")

score = cross_val_score(modello, X_train, y_train, cv = 10)
print("Cross validation: \n",score)
print("Testing score: {}".format(modello.score(X_test, y_test)))
print("\n")

In [253]:
#@title Download txt utenti e formattazione in dataset
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/FiddiCoder/-FIA-Project/main/"
FILE_PATH = os.path.join("dataset")
FILE_URL1 = DOWNLOAD_ROOT + "dataset/users.user"

def fetch_file_data1(file_url1=FILE_URL1, file_path1=FILE_PATH):
    os.makedirs(file_path1, exist_ok=True)
    csv_path1 = os.path.join(file_path1, "users.user")
    urllib.request.urlretrieve(file_url1, csv_path1)

fetch_file_data1()
datapath = os.path.join("dataset","")

userCols = ['user_id', 'age', 'gender', 'occupation', 'zip_code']
users = pd.read_csv('./dataset/users.user', sep='|', names=userCols)


In [ ]:
#@title Data cleaning, trasformazione variabili da string a boolean

#Drop feature inutile
users = users.drop('zip_code', axis=1)

#Data cleaning = Eliminazione righe vuote
users = users[users['gender'] != 'empty']
users = users[users['occupation'] != 'empty']

#Colonne occupation e gender trasformate in più colonne coi rispettivi nomi delle variabili per dare loro valori booleani
users['occupation'] = users['occupation'].astype('category')
users['gender'] = users['gender'].astype('category')
one_hot_occ = users.occupation.str.get_dummies()
users = users.drop('occupation',axis=1)
users = users.join(one_hot_occ,how='inner')
one_hot_g = users.gender.str.get_dummies()
users = users.drop('gender',axis=1)
users = users.join(one_hot_g,how='inner')
users.head(5)

In [ ]:
#@title Download dataset chiave per joinare utenti e film
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/FiddiCoder/-FIA-Project/main/"
FILE_PATH = os.path.join("dataset")
FILE_URL1 = DOWNLOAD_ROOT + "dataset/usersmovies.data"

def fetch_file_data1(file_url1=FILE_URL1, file_path1=FILE_PATH):
    os.makedirs(file_path1, exist_ok=True)
    csv_path1 = os.path.join(file_path1, "usersmovies.data")
    urllib.request.urlretrieve(file_url1, csv_path1)

fetch_file_data1()
datapath = os.path.join("dataset","")

colonnekey = ['user_id', 'movie_id', 'rating', 'timestamp']
key = pd.read_csv('./dataset/usersmovies.data', sep='\t', names=colonnekey)

#Drop feature inutile
key = key.drop('timestamp', axis=1)
key.head(50)

In [256]:
#@title Drop vecchia colonna target 'like', da ora utilizzeremo 'rating'
movies = movies.drop('like', axis=1)

In [ ]:
#@title Join colonne utenti e film
final_dataset = key.merge(movies, left_on='movie_id', right_on='movie_id', how='inner')
final_dataset = final_dataset.merge(users, left_on='user_id', right_on='user_id', how='inner')

#Spostiamo la colonna target in ultima posizione
sposta_colonna = final_dataset.pop("rating")
final_dataset.insert(46, "rating", sposta_colonna)

final_dataset.head(10)

In [ ]:
#@title Secondo Test: Decision tree
#@markdown Risultati ancora scarsi, se non peggiori.

#@markdown Abbiamo di conseguenza pensato di trasformare la variabile target in un booleano, affinchè potesse risultare più semplice la classificazione tramite decision tree
X = final_dataset.iloc[:,3:46]
print(X.columns)
y = final_dataset.iloc[:,46:]
print(y.columns)
X_train, X_test, y_train, y_test = train_test_split(X, y,
test_size=0.20, random_state=0)

dtree = tree.DecisionTreeClassifier()
modello = dtree.fit(X_train.values, y_train)

for i in range(1, len(final_dataset)):
    movie = final_dataset.iloc[i, 2]
    if(movie == 'Raising Arizona (1987)'):
        movie_1_ind = final_dataset.iloc[i, 3:46]
    if(movie == 'Little Big League (1994)'):
        movie_2_ind = final_dataset.iloc[i, 3:46]
    if (movie == 'Mighty Aphrodite (1995)'):
        movie_3_ind = final_dataset.iloc[i, 3:46]
print("prediction for Raising Arizona (1987))")
predict = dtree.predict([movie_1_ind])
print(predict)
print("prediction for Dead Man Walking (1995)")
predict = dtree.predict([movie_2_ind])
print(predict)
print("prediction for Chasing Amy (1997)")
predict = dtree.predict([movie_3_ind])
print(predict)
print("\n")


score = cross_val_score(modello, X_train, y_train, cv = 10)
print("cross validation scores: \n",score)
print("testing score: {}".format(modello.score(X_test, y_test)))
print("\n")


predict = dtree.predict(X_test)
print('Accuracy modello: {0:0.4f}'.
format(accuracy_score(y_test, predict)))

cm = confusion_matrix(y_test, predict)
print('Confusion matrix\n\n', cm)

In [ ]:
#@title Trasformazione colonna target in booleana
final_dataset['rating'] = final_dataset['rating'].replace(0,0)
final_dataset['rating'] = final_dataset['rating'].replace(1,0)
final_dataset['rating'] = final_dataset['rating'].replace(2,0)
final_dataset['rating'] = final_dataset['rating'].replace(3,1)
final_dataset['rating'] = final_dataset['rating'].replace(4,1)
final_dataset['rating'] = final_dataset['rating'].replace(5,1)
final_dataset.head(10)

In [ ]:
#@title Terzo Test: Decision tree
#@markdown Risultati considerevolmente migliori.
#@markdown
#@markdown Abbiamo pensato di fare Feature Selection per migliorare ulteriormente lo score.
X = final_dataset.iloc[:,3:46]
print(X.columns)
y = final_dataset.iloc[:,46:]
print(y.columns)
X_train, X_test, y_train, y_test = train_test_split(X, y,
test_size=0.20, random_state=0)

dtree = tree.DecisionTreeClassifier()
modello = dtree.fit(X_train.values, y_train)

for i in range(1, len(final_dataset)):
    movie = final_dataset.iloc[i, 2]
    if(movie == 'Raising Arizona (1987)'):
        movie_1_ind = final_dataset.iloc[i, 3:46]
    if(movie == 'Little Big League (1994)'):
        movie_2_ind = final_dataset.iloc[i, 3:46]
    if (movie == 'Mighty Aphrodite (1995)'):
        movie_3_ind = final_dataset.iloc[i, 3:46]
print("prediction for Raising Arizona (1987))")
predict = dtree.predict([movie_1_ind])
print(predict)
print("prediction for Dead Man Walking (1995)")
predict = dtree.predict([movie_2_ind])
print(predict)
print("prediction for Chasing Amy (1997)")
predict = dtree.predict([movie_3_ind])
print(predict)
print("\n")


score = cross_val_score(modello, X_train, y_train, cv = 10)
print("Cross validation: \n",score)
print("Testing score: {}".format(modello.score(X_test, y_test)))
print("\n")


predict = dtree.predict(X_test)
print('Accuracy modello: {0:0.4f}'.
format(accuracy_score(y_test, predict)))

cm = confusion_matrix(y_test, predict)
print('Confusion matrix\n\n', cm)
